In [1]:
import os
import pandas as pd
import json
import requests
import elasticsearch
from elasticsearch import helpers

In [2]:
base_path = os.getcwd() + "/aiml_fanzfeng/data/"
data = pd.read_excel(base_path+"行政区划清单 V3.0 9.03.xlsx")
data.head()

,ID,省、直辖市名称,城市名称,区域名称,省GB码,市GB码,参考GB码,别名
0,1,北京市,北京市,北京市,NaN,NaN,110000,NaN
1,2,北京市,北京市,朝阳区,110000.0,110100.0,110105,NaN
2,3,北京市,北京市,海淀区,110000.0,110100.0,110108,NaN
3,4,北京市,北京市,房山区,110000.0,110100.0,110111,NaN
4,5,北京市,北京市,顺义区,110000.0,110100.0,110113,NaN


In [10]:
len(data), sum(data[u"城市名称"] == data[u"区域名称"])

(3222, 392, (3175,))

In [31]:
data["isna"] = pd.isnull(data[u"省GB码"])
data[data[u"城市名称"] == u"上海市"]
# pd.isnull(data.loc[789, u"省GB码"])
df = data[~data["isna"]]

In [34]:
df[df[u"城市名称"] == df[u"区域名称"]][u"区域名称"].value_counts()
# data.head()
df[df[u"城市名称"] == u"潜江市"]

,ID,省、直辖市名称,城市名称,区域名称,省GB码,市GB码,参考GB码,别名,isna
1790,1791,湖北省,潜江市,潜江市,420000.0,NaN,429005,NaN,False
1792,1793,湖北省,潜江市,潜江市,420000.0,429005.0,429005,NaN,False


In [35]:
data[u"市GB码"].unique().shape, data[u"城市名称"].unique().shape

((367,), (392,))

In [38]:
sum(data[u"省、直辖市名称"] == data[u"城市名称"])
dfg = data[[u"省、直辖市名称", u"参考GB码"]]
dfg.drop_duplicates().shape, dfg[u"参考GB码"].unique().shape
dfg.head()

,省、直辖市名称,参考GB码
0,北京市,110000
1,北京市,110105
2,北京市,110108
3,北京市,110111
4,北京市,110113


In [8]:
data.columns

Index([u'ID', u'省、直辖市名称', u'城市名称', u'区域名称', u'省GB码', u'市GB码', u'参考GB码', u'别名'], dtype='object')

In [3]:
dd = data[data[u"省GB码"].apply(lambda x: pd.isnull(x))][[u'省、直辖市名称', u'参考GB码']]
dd.columns = ["area", "code"]
dd_dict = dd.to_dict("record")

In [4]:
dd_dict

[{'area': '北京市', 'code': 110000},
 {'area': '天津市', 'code': 120000},
 {'area': '河北省', 'code': 130000},
 {'area': '山西省', 'code': 140000},
 {'area': '内蒙古自治区', 'code': 150000},
 {'area': '辽宁省', 'code': 210000},
 {'area': '吉林省', 'code': 220000},
 {'area': '黑龙江省', 'code': 230000},
 {'area': '上海市', 'code': 310000},
 {'area': '江苏省', 'code': 320000},
 {'area': '浙江省', 'code': 330000},
 {'area': '安徽省', 'code': 340000},
 {'area': '福建省', 'code': 350000},
 {'area': '江西省', 'code': 360000},
 {'area': '山东省', 'code': 370000},
 {'area': '河南省', 'code': 410000},
 {'area': '湖北省', 'code': 420000},
 {'area': '湖南省', 'code': 430000},
 {'area': '广东省', 'code': 440000},
 {'area': '广西壮族自治区', 'code': 450000},
 {'area': '海南省', 'code': 460000},
 {'area': '重庆市', 'code': 500000},
 {'area': '四川省', 'code': 510000},
 {'area': '贵州省', 'code': 520000},
 {'area': '云南省', 'code': 530000},
 {'area': '西藏自治区', 'code': 540000},
 {'area': '陕西省', 'code': 610000},
 {'area': '甘肃省', 'code': 620000},
 {'area': '青海省', 'code': 630000},
 {'a

## 高德地域数据导入es

In [5]:
ip = "101.200.61.56"
es = elasticsearch.Elasticsearch([ip], http_auth=('elastic', 'changeme'))
es.info

<bound method Elasticsearch.info of <Elasticsearch([{'host': '101.200.61.56'}])>>

In [6]:
list_dicts = [{"_index": "xiaopang7", "_type": "province",
               "_source": s} for s in dd_dict]
helpers.bulk(es, list_dicts)
print("ok")

ok


In [13]:
es.search(index="xiaopang7", doc_type="province", 
          body={"query": {"match": {"area": "天津天气"}}})['hits']['hits']


[{'_id': 'AWTmfQZIfiZKEn7D9sSj',
  '_index': 'xiaopang7',
  '_score': 3.2311888,
  '_source': {'area': '天津市', 'code': 120000},
  '_type': 'province'}]

In [ ]:
def isChinese(c):
    r = [
        # 标准CJK文字
        (0x3400, 0x4DB5), (0x4E00, 0x9FA5), (0x9FA6, 0x9FBB), (0xF900, 0xFA2D),
        (0xFA30, 0xFA6A), (0xFA70, 0xFAD9), (0x20000, 0x2A6D6), (0x2F800, 0x2FA1D),
        # 全角ASCII、全角中英文标点、半宽片假名、半宽平假名、半宽韩文字母
        # (0xFF00, 0xFFEF),
        # # CJK部首补充
        # (0x2E80, 0x2EFF),
        # # CJK标点符号
        # (0x3000, 0x303F),
        # # CJK笔划
        # (0x31C0, 0x31EF)
    ]
    return any(s <= ord(c) <= e for s, e in r)
isChinese("我")

True